# S02: 特徴量エンジニアリング

このノートブックでは以下の処理を行います：
1. 全HTMLファイルをパースして中間ファイル（CSV）を生成
2. **差分更新**: 新規HTMLのみを処理して既存CSVに追加

**実行環境**: Sagemaker

---
## 1. 環境設定

In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルート
PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# データパス
HTML_DIR = PROJECT_ROOT / 'data' / 'raceHTML'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
FEATURES_DIR = PROJECT_ROOT / 'data' / 'features'

# ディレクトリ作成
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

# ファイルパス
RACES_CSV = PROCESSED_DIR / 'races.csv'
RESULTS_CSV = PROCESSED_DIR / 'results.csv'

print(f"HTML_DIR: {HTML_DIR.resolve()}")
print(f"RACES_CSV: {RACES_CSV}")
print(f"RESULTS_CSV: {RESULTS_CSV}")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# モジュールインポート
from scraper.parser import parse_incremental, parse_multiple_html_full, classify_race_level

print("✅ モジュール読み込み完了")

---
## 2. 差分更新（推奨）

既存のCSVがある場合、新規HTMLのみを処理して追加します。
初回実行時は全HTMLを処理します。

In [ ]:
# 現在の状態を確認
if RACES_CSV.exists():
    existing_races = pd.read_csv(RACES_CSV)
    existing_results = pd.read_csv(RESULTS_CSV)
    print(f"📊 既存データ:")
    print(f"   レース数: {len(existing_races):,}")
    print(f"   出走馬データ数: {len(existing_results):,}")
    print(f"   races.csv: {RACES_CSV.stat().st_size / 1024 / 1024:.2f} MB")
    print(f"   results.csv: {RESULTS_CSV.stat().st_size / 1024 / 1024:.2f} MB")
else:
    print("中間ファイルが見つかりません。初回作成を行います。")

In [ ]:
# 差分更新実行
def progress_callback(processed, total, skipped=0):
    new_count = processed - skipped
    print(f"\r進捗: {processed}/{total} (スキップ: {skipped}, 新規: {new_count})", end='')

print("📥 差分更新を開始します...")
print(f"対象: {HTML_DIR}")

new_races, new_horses = parse_incremental(
    HTML_DIR,
    RACES_CSV,
    RESULTS_CSV,
    years=None,  # 全年をチェック
    progress_callback=progress_callback
)

print(f"\n\n✅ 差分更新完了")
print(f"   新規レース: {new_races:,}")
print(f"   新規出走馬データ: {new_horses:,}")

---
## 3. 全件再生成（必要な場合のみ）

パーサーを更新した場合など、全HTMLを再パースしたい場合に実行

In [ ]:
# 全件再生成する場合はこのセルのコメントを外して実行
# 警告: 時間がかかります

# def progress_full(processed, total, skipped=0):
#     print(f"\r進捗: {processed}/{total} ({processed/total*100:.1f}%)", end='')

# print("⚠️ 全HTMLを再パースします...")

# # 既存ファイルを削除
# if RACES_CSV.exists():
#     RACES_CSV.unlink()
# if RESULTS_CSV.exists():
#     RESULTS_CSV.unlink()

# # 全件パース
# race_list, horse_list = parse_multiple_html_full(
#     HTML_DIR,
#     years=None,
#     progress_callback=progress_full
# )

# # 保存
# pd.DataFrame(race_list).to_csv(RACES_CSV, index=False, encoding='utf-8')
# pd.DataFrame(horse_list).to_csv(RESULTS_CSV, index=False, encoding='utf-8')

# print(f"\n\n✅ 全件再生成完了")
# print(f"   レース数: {len(race_list):,}")
# print(f"   出走馬データ数: {len(horse_list):,}")

---
## 4. データ確認

In [ ]:
# 最新状態を確認
races_df = pd.read_csv(RACES_CSV)
results_df = pd.read_csv(RESULTS_CSV)

print("=== 最新データ統計 ===")
print(f"レース数: {len(races_df):,}")
print(f"出走馬データ数: {len(results_df):,}")

print(f"\n=== レースデータ カラム ({len(races_df.columns)}) ===")
print(races_df.columns.tolist())

print(f"\n=== 出走馬データ カラム ({len(results_df.columns)}) ===")
print(results_df.columns.tolist())

In [ ]:
# 年別レース数
if 'year' in races_df.columns:
    print("=== 年別レース数 ===")
    year_counts = races_df.groupby('year').size()
    print(year_counts.to_string())

In [ ]:
# レースレベル分布
if 'race_level' in races_df.columns:
    print("=== レースレベル分布 ===")
    level_counts = races_df['race_level'].value_counts().sort_index()
    print(level_counts.to_string())

In [ ]:
# サンプルデータ
print("=== レースデータ サンプル ===")
display(races_df.tail(5))

In [ ]:
# サンプルデータ
print("=== 出走馬データ サンプル ===")
display(results_df.tail(5))

---
## 5. 抽出項目一覧

### レース情報 (races.csv)

| カラム | 説明 |
|--------|------|
| race_id | レースID（12桁） |
| venue_code | 競馬場コード |
| venue_name | 競馬場名 |
| kaisai | 開催回 |
| day | 開催日 |
| race_num | レース番号 |
| race_name | レース名 |
| date | 日付 |
| year | 年 |
| month | 月 |
| surface | 芝/ダート |
| distance | 距離 |
| direction | 右/左回り |
| weather | 天候 |
| track_condition | 馬場状態 |
| start_time | 発走時刻 |
| horse_count | 出走頭数 |
| first_prize | 1着賞金 |
| race_level | レースレベル |
| level_score | レベルスコア |

### 出走馬データ (results.csv)

| カラム | 説明 |
|--------|------|
| race_id | レースID |
| horse_id | 馬ID |
| horse_name | 馬名 |
| finish_position | 着順 |
| is_finished | 完走フラグ |
| dnf_reason | 失格理由 |
| gate_number | 枠番 |
| horse_number | 馬番 |
| sex | 性別 |
| age | 年齢 |
| impost | 斤量 |
| jockey_id | 騎手ID |
| jockey_name | 騎手名 |
| trainer_id | 調教師ID |
| trainer_name | 調教師名 |
| trainer_region | 調教師所属（東/西） |
| owner_id | 馬主ID |
| owner_name | 馬主名 |
| time_str | タイム（文字列） |
| time_seconds | タイム（秒） |
| margin | 着差 |
| passing_order | 通過順 |
| first_corner | 1コーナー順位 |
| last_corner | 最終コーナー順位 |
| last_3f | 上がり3F |
| odds | 単勝オッズ |
| popularity | 人気 |
| horse_weight | 馬体重 |
| weight_change | 馬体重増減 |
| prize_money | 獲得賞金 |
| race_level | レースレベル |
| level_score | レベルスコア |
| race_date | レース日 |
| venue_code | 競馬場コード |
| venue_name | 競馬場名 |
| distance | 距離 |
| surface | 芝/ダート |
| track_condition | 馬場状態 |